In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from tqdm import tqdm
import spacy
from gensim import corpora, models

import sys
sys.path.append('..')
from utils import preprocess, get_windows

In [3]:
MIN_COUNTS = 20
MAX_COUNTS = 1800
# words with count < MIN_COUNTS
# and count > MAX_COUNTS
# will be removed

MIN_LENGTH = 15
# minimum document length 
# (number of words)
# after preprocessing

# half the size of the context around a word
HALF_WINDOW_SIZE = 5
# it must be that 2*HALF_WINDOW_SIZE < MIN_LENGTH

# Load NLP model

In [4]:
nlp = spacy.load('en')

# Load dataset

In [5]:
dataset = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
docs = dataset['data']

In [6]:
# number of documents
len(docs)

18846

In [7]:
# store an index with a document
docs = [(i, doc) for i, doc in enumerate(docs)]

# Preprocess dataset and create windows

In [8]:
encoded_docs, decoder, word_counts = preprocess(
    docs, nlp, MIN_LENGTH, MIN_COUNTS, MAX_COUNTS
)

100%|██████████| 18846/18846 [00:47<00:00, 397.27it/s]


number of removed short documents: 3985
total number of tokens: 1439861
number of tokens to be removed: 393091
number of additionally removed short documents: 2032
total number of tokens: 1023189

minimum word count number: 14
this number can be less than MIN_COUNTS because of document removal


In [9]:
# new ids will be created for the documents.
# create a way of restoring initial ids:
doc_decoder = {i: doc_id for i, (doc_id, doc) in enumerate(encoded_docs)}

In [10]:
data = []
# new ids are created here
for index, (_, doc) in tqdm(enumerate(encoded_docs)):
    windows = get_windows(doc, HALF_WINDOW_SIZE)
    # index represents id of a document, 
    # windows is a list of (word, window around this word),
    # where word is in the document
    data += [[index, w[0]] + w[1] for w in windows]

data = np.array(data, dtype='int64')

12829it [00:02, 4569.99it/s]


In [11]:
# a row in 'data' contains:
# id of a document, id of a word in this document, a window around this word
# 1 + 1 + 10
data.shape[1]

12

In [12]:
# number of windows (equals to the total number of tokens)
data.shape[0]

1023189

# Get unigram distribution

In [13]:
word_counts = np.array(word_counts)
unigram_distribution = word_counts/sum(word_counts)

# Prepare word vectors

In [14]:
%%time
vocab_size = len(decoder)
embedding_dim = 50

# train a skip-gram word2vec model
texts = [[str(j) for j in doc] for i, doc in encoded_docs]
model = models.Word2Vec(texts, size=embedding_dim, window=5, workers=4, sg=1, negative=15, iter=70)
model.init_sims(replace=True)

word_vectors = np.zeros((vocab_size, embedding_dim)).astype('float32')
for i in decoder:
    word_vectors[i] = model.wv[str(i)]

CPU times: user 34min 27s, sys: 1.95 s, total: 34min 29s
Wall time: 8min 42s


In [15]:
# number of unique words
vocab_size

7460

# Prepare initialization for document weights

In [16]:
texts = [[decoder[j] for j in doc] for i, doc in encoded_docs]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [17]:
%%time
n_topics = 25
lda = models.LdaModel(corpus, alpha=0.9, id2word=dictionary, num_topics=n_topics)
corpus_lda = lda[corpus]

CPU times: user 2min 24s, sys: 1.69 s, total: 2min 26s
Wall time: 1min 22s


In [18]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i, ':', ' '.join([t for t, _ in topics]))

topic 0 : drug lot job ordnance school kid food young little buy
topic 1 : atheist belief religion exist argument evidence atheism reason religious true
topic 2 : fbi fire weapon police crime koresh batf gun place criminal
topic 3 : card disk board mode port video software controller speed cpu
topic 4 : team play win player season hockey league goal period score
topic 5 : color section version character software font display type mac disk
topic 6 : science planet earth universe theory book space shall scientific amendment
topic 7 : modem dog signal john baltimore springfield moncton rochester radar providence
topic 8 : cover appear book copy black green price rider bag force
topic 9 : university history turkey armenian war muslim iran jews source press
topic 10 : israel kill armenians armenian jews israeli leave turkish attack child
topic 11 : jesus church bible word christian christ life sin lord love
topic 12 : dos windows network window mouse user newsgroup microsoft driver access
t

In [19]:
doc_weights_init = np.zeros((len(corpus_lda), n_topics))
for i in tqdm(range(len(corpus_lda))):
    topics = corpus_lda[i]
    for j, prob in topics:
        doc_weights_init[i, j] = prob

100%|██████████| 12829/12829 [01:17<00:00, 166.33it/s]


# Save data

In [20]:
np.save('data.npy', data)
np.save('word_vectors.npy', word_vectors)
np.save('unigram_distribution.npy', unigram_distribution)
np.save('decoder.npy', decoder)
np.save('doc_decoder.npy', doc_decoder)
np.save('doc_weights_init.npy', doc_weights_init)